In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using Theano backend.
Using gpu device 0: GeForce 920M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5005)


In [2]:
batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

15302656/15296311 [==============================] - 17s    


In [3]:
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
print(Y_train, 'Y_train')
print(Y_test, 'Y_test')

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]] Y_train
[[ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] Y_test


In [4]:
model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [6]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 42s - loss: 0.3842 - acc: 0.8808 - val_loss: 0.0966 - val_acc: 0.9700
Epoch 2/12
60000/60000 [==============================] - 41s - loss: 0.1528 - acc: 0.9546 - val_loss: 0.0667 - val_acc: 0.9787
Epoch 3/12
60000/60000 [==============================] - 41s - loss: 0.1168 - acc: 0.9657 - val_loss: 0.0557 - val_acc: 0.9821
Epoch 4/12
60000/60000 [==============================] - 41s - loss: 0.0975 - acc: 0.9707 - val_loss: 0.0495 - val_acc: 0.9834
Epoch 5/12
60000/60000 [==============================] - 41s - loss: 0.0842 - acc: 0.9746 - val_loss: 0.0433 - val_acc: 0.9858
Epoch 6/12
60000/60000 [==============================] - 41s - loss: 0.0761 - acc: 0.9773 - val_loss: 0.0407 - val_acc: 0.9864
Epoch 7/12
60000/60000 [==============================] - 41s - loss: 0.0695 - acc: 0.9796 - val_loss: 0.0367 - val_acc: 0.9870
Epoch 8/12
60000/60000 [==============================

In [7]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0313120700188
Test accuracy: 0.9887


In [10]:
import json
json_string = model.to_json()
open('mnist_gpu.json', 'w').write(json_string)
model.save_weights('mnist_gpu_weights.h5')